## This model constuct a networkx graph from data using the following nodes

### Nodes:
- City: aggregate all stations within a city

### Edges:
- BikeTrip: bike trip between cities

In [ ]:
import pandas as pd
import networkx as nx
from graphgen import fix_columns, create_graph

In [ ]:
trips_filename = '../data//201508_trip_data.csv'
stations_filename = '../data/201508_station_data.csv'

trips_df = pd.read_csv(trips_filename)
stations_df = pd.read_csv(stations_filename)

# if columns have spaces in their names we need to replace them with underscore
fix_columns(trips_df)
fix_columns(stations_df)

In [ ]:
display(stations_df.head())

In [ ]:
display(stations_df['landmark'].unique())

In [ ]:
trips_df.loc[0,'Start_Terminal']

In [ ]:
display(stations_df[stations_df.station_id == trips_df.loc[0, 'Start_Terminal']]['landmark'])

In [ ]:
city_df = stations_df[['station_id', 'landmark']]
city_df.set_index('station_id', inplace=True)

In [ ]:
# city_df.set_index('station_id', inplace=True)
landmark_dict = city_df['landmark'].to_dict()

In [ ]:
station_mapper = {
    'nodes': [
        {
            'type'      : 'City',
            'key'       : 'city_id',
            'attributes': ['landmark']
        },
    ]
}

edges_mapper = {
    'edges': [
        {
            'type'      : 'BikeTrip',
            'from'      : {
                'key'     :'city_id',
                'node_key': 'city_id'
            },
            'to'        : {
                'key'     : 'city_id',
                'node_key': 'city_id'
            },
            'attributes': ['Trip_ID', 'Start_Date', 'End_Date', 'Duration']
        }
    ]
}